In [2]:
import $ivy.`org.apache.spark::spark-sql:2.4.7`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

val spark = NotebookSparkSession
    .builder()
    .master("local[*]")
    .getOrCreate()

Getting spark JARs
Creating SparkSession


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@7cdcf0a7

In [3]:
def sc = spark.sparkContext

val rdd = sc.parallelize(1 to 100000000, 100)

val n = rdd.map(x => x + 1).filter(x => x > 100).reduce((x, y) => x + y)

// 0 1 2 3 4
// map
// 1 2 3 4 5
// reduce(f)
// f(1, 2) f(3, 4) 5
// 3 7 5
// f(3, 7) 5
// 10 5
// f(10, 5)
// 15

reduce at cmd2.sc:5

100 / 100

reduce at cmd2.sc:5

100 / 100

defined function sc
rdd: org.apache.spark.rdd.RDD[Int] = ParallelCollectionRDD[0] at parallelize at cmd2.sc:3
n: Int = 1087454663

In [ ]:
val lines = sc.textFile("data/crime.csv")

val lineLengths = lines.map(s => s.length)

val totalLength = lineLengths.reduce((a, b) => a + b)

In [4]:
val lines = sc.textFile("data/crime.csv").cache()

val totalLength = lines
    .map(s => s.length)
    .reduce((a, b) => a + b)

val counts = lines
    .flatMap(x => x.split(","))
    .map(s => (s, 1))     // RDD[(key, value)]
    .reduceByKey((a, b) => a + b)
    .sortBy(x => -x._2)
    .take(10)

reduce at cmd3.sc:5

2 / 2

reduce at cmd3.sc:5

2 / 2

map at cmd3.sc:9

...

take at cmd3.sc:12

1 / 1

take at cmd3.sc:12

1 / 1

lines: org.apache.spark.rdd.RDD[String] = data/crime.csv MapPartitionsRDD[4] at textFile at cmd3.sc:1
totalLength: Int = 57331362
counts: Array[(String, Int)] = Array(
  ("", 370778),
  ("Part Three", 158553),
  ("2017", 100886),
  ("2016", 99114),
  ("Part Two", 97569),
  ("2018", 65685),
  ("Part One", 61629),
  ("2015", 53388),
  ("B2", 49945),
  ("Friday", 48495)
)

In [5]:
val crimeFacts = spark
    .read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv("data/crime.csv")

crimeFacts.show

csv at cmd4.sc:5

1 / 1

csv at cmd4.sc:5

1 / 1

csv at cmd4.sc:5

...

1 / 1

show at cmd4.sc:7

1 / 1

+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------------+-----------+------------+--------------------+
|INCIDENT_NUMBER|OFFENSE_CODE|  OFFENSE_CODE_GROUP| OFFENSE_DESCRIPTION|DISTRICT|REPORTING_AREA|SHOOTING|   OCCURRED_ON_DATE|YEAR|MONTH|DAY_OF_WEEK|HOUR|  UCR_PART|           STREET|        Lat|        Long|            Location|
+---------------+------------+--------------------+--------------------+--------+--------------+--------+-------------------+----+-----+-----------+----+----------+-----------------+-----------+------------+--------------------+
|     I182070945|         619|             Larceny|  LARCENY ALL OTHERS|     D14|           808|    null|2018-09-02 13:00:00|2018|    9|     Sunday|  13|  Part One|       LINCOLN ST|42.35779134|-71.13937053|(42.35779134, -71...|
|     I182070943|        1402|           Vandalism|           VANDALISM|     C11|   

crimeFacts: DataFrame = [INCIDENT_NUMBER: string, OFFENSE_CODE: int ... 15 more fields]

In [ ]:
crimeFacts.printSchema

In [ ]:
import spark.implicits._

$"OFFENSE_CODE" + 1
"OFFENSE_CODE" + 1
// crimeFacts.select($"OFFENSE_CODE" + 1)
// crimeFacts.select("OFFENSE_CODE" + 1)

In [ ]:
crimeFacts.createOrReplaceTempView("crimes")

In [ ]:
spark.sql("select INCIDENT_NUMBER, DISTRICT from crimes limit 10").show

In [ ]:
import spark.implicits._

org.apache.spark.sql.catalyst.encoders.OuterScopes.addOuterScope(this) // Almond bug

case class Crime (
    INCIDENT_NUMBER: Option[String],
    OFFENSE_CODE: Option[Int],
    OFFENSE_CODE_GROUP: Option[String],
    OFFENSE_DESCRIPTION: Option[String],
    DISTRICT: Option[String],
    REPORTING_AREA: Option[String],
    SHOOTING: Option[String],
    OCCURRED_ON_DATE: Option[String],
    YEAR: Option[Int],
    MONTH: Option[Int],
    DAY_OF_WEEK: Option[String],
    HOUR: Option[Int],
    UCR_PART: Option[String],
    STREET: Option[String],
    Lat: Option[Double],
    Long: Option[Double],
    Location: Option[String]
) {
    
    def wasShooting: Boolean = { SHOOTING.nonEmpty }
    
}

crimeFacts.as[Crime].filter(x => x.OFFENSE_DESCRIPTION == Some("VANDALISM")).show
crimeFacts.as[Crime].filter(x => x.wasShooting).show
crimeFacts.as[Crime].take(10).foreach(x => println(x.SHOOTING))

In [ ]:
val offenseCodes = spark.read.option("header", "true").option("inferSchema", "true").csv("data/offense_codes.csv")
offenseCodes.show(false)

In [ ]:
import spark.implicits._

crimeFacts
    .join(offenseCodes, $"CODE" === $"OFFENSE_CODE")
    .where($"NAME".startsWith("ROBBERY"))
    .groupBy($"NAME")
    .count()
    .orderBy($"count".desc)
    .show(false)

In [ ]:
import org.apache.spark.sql.functions.broadcast

val offenseCodesBroadcast = broadcast(offenseCodes)

val robberyStatsWithBroadcast = crimeFacts
    .join(offenseCodesBroadcast, $"CODE" === $"OFFENSE_CODE")
    .filter($"NAME".startsWith("ROBBERY"))
    .groupBy($"NAME")
    .count()
    .orderBy($"count".desc)
    
robberyStatsWithBroadcast.show

In [ ]:
// udf